In [10]:
# import boyer_more helper
from bm_preproc import BoyerMoore
from kmer_index import Index
import bisect
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            mismatches = 0
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [34]:
# naive 
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

# naive with count 
# naive 
def naive_with_counts(p, t):
    occurrences = []
    alignComps = 0
    chrComps = 0 
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        alignComps += 1 
        match = True
        for j in range(len(p)):  # loop over characters
            chrComps += 1 
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences, alignComps, chrComps

# boyer_moore
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences


# boyer_moore with counts 
def boyer_moore_with_counts(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    alignComps = 0
    chrComps = 0 
    while i < len(t) - len(p) + 1:
        alignComps += 1 
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            chrComps += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, alignComps, chrComps

# read genome 
# - filename: the file location of the genome file to read
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome
def naive_2mm(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        miss = 0 
        for j in range(len(p)): 
            if t[i+j] != p[j]:  # compare characters
                miss = miss + 1
                if miss > 2: 
                    match = False 
                    break 
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences


def approximate_match_boyer(p, t, n):
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()
    hits = 0
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        p_bm = BoyerMoore(p[start:end], alphabet='ACGT')
        matches = boyer_moore(p[start:end], p_bm, t)
        hits += len(matches)
        # Extend matching segments to see if whole p matches
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches), hits 


def approximate_match_index(p, index, t, n):
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()
    hits = 0 
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        matches = index.query(p[start:end])
        hits += len(matches)
        # Extend matching segments to see if whole p matches
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches), hits 


   

def approximate_match_index_sub(p, index, t, n):
    all_matches = set()
    hits = 0 
    ivals = index.ival
    for i in range(ivals):
        start = i
        end = len(p)
        matches = index.query(p[start:end])
        hits += len(matches)
        # Extend matching segments to see if whole p matches
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches), hits 

In [ ]:
!curl http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta -o C:/Users/buehl/Documents/genomics/alg4Genom/hw2/lambd_virus.fa

In [ ]:
# test case 1 naive_with_counts
p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

In [ ]:
# test case 2 naive_with_counts
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

In [ ]:
# test case 2 naive_with_counts
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

In [ ]:
# test case 2 naive_with_counts
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

In [ ]:
# Question 1 
# define text 
t = readGenome("lambd_virus.fa")
p = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
lowercase_alphabet = 'ACGT'
# run 
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

In [ ]:
# Question 2-3
# define text 
t = readGenome("lambd_virus.fa")
p = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
lowercase_alphabet = 'ACGT'
# pre-process
p_bm = BoyerMoore(p, lowercase_alphabet)
# run 
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

In [19]:
# Question 4
t = readGenome("lambd_virus.fa")
p = "GGCGCGGTGGCTCACGCCTGTAAT"
index = Index(t, 8)
matches, hits = approximate_match_index(p, index, t, 2)
print(matches)
print(hits)
matches, hits = approximate_match_boyer(p, t, 2)
print(matches)
print(hits)
matches = naive_2mm(p, t)
print(len(matches))

[273669, 681737, 717706, 262042, 635931, 84641, 160162, 724927, 657496, 160729, 56922, 191452, 551134, 747359, 421221, 147558, 364263, 465647, 429299]
90
[273669, 681737, 717706, 262042, 635931, 84641, 160162, 724927, 657496, 160729, 56922, 191452, 551134, 747359, 421221, 147558, 364263, 465647, 429299]
90
19


In [37]:
# Question 5 understanding 
ind = SubseqIndex('ATATAT', 3, 2)
print(ind.index)
p = 'TTATAT'
print(ind.query(p[0:]))
print(ind.query(p[1:]))
print(p[1:])

[('AAA', 0), ('TTT', 1)]
[]
[1]
TATAT


In [35]:
# Question 5 tests
t = 'to-morrow and to-morrow and to-morrow creeps in this petty pace'
p = 'to-morrow and to-morrow '
subseq_ind = SubseqIndex(t, 8, 3)
occurrences, num_index_hits = approximate_match_index_sub(p, subseq_ind,t,2)
print(occurrences)
print(num_index_hits)

[0, 14]
6


In [36]:
# Question 5 
t = readGenome("lambd_virus.fa")
p = "GGCGCGGTGGCTCACGCCTGTAAT"
index = SubseqIndex(t, 8, 3)
# print(index.k)
matches, hits = approximate_match_index_sub(p, index, t, 2)
print(matches)
print(hits)

[712449, 651523, 558456, 273669, 349189, 681737, 717706, 658702, 794642, 551827, 83863, 579737, 262042, 635931, 471966, 262174, 67486, 84641, 596898, 160162, 151718, 84775, 745640, 100011, 322735, 479030, 199607, 472634, 489019, 454332, 719418, 724927, 719557, 108110, 707151, 766419, 657496, 160729, 56922, 23003, 191452, 704731, 551134, 747359, 366818, 421221, 147558, 364263, 137574, 421354, 465647, 783346, 429299, 141046, 124024, 746620, 22397, 32639]
79
